In [157]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *

'''
    LIAR
'''



data = pd.read_csv(r"../data/Des_fake_news/LIAR_PROCESSED_2.csv")



In [158]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd

def tf_idf_vectorize(df: pd.DataFrame, corpus: pd.Series, vocabulary: list[str]) -> tuple[list[str], pd.DataFrame]:
    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=1000, min_df=50, ngram_range=(1,3), vocabulary=vocabulary)
    features = vectorizer.fit_transform(corpus).toarray()
    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(features, columns=headers)
    #final = pd.concat([df, feature_frame], axis=1)
    return (names, feature_frame)


def get_full_vocabulary(corpus: pd.Series) -> list[str]:
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stopwords.words("english"), max_features=1000, min_df=50, ngram_range=(1,3))
    vectorizer.fit_transform(corpus).toarray()
    return vectorizer.get_feature_names_out()


def train_test_split_and_vectorize(x: pd.DataFrame, y: pd.DataFrame, corpus: pd.Series, test_size=0.15, random_state=42) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    temp = pd.concat((x, corpus), axis=1)
    # first, we need a vocabulary from the entire dataset
    vocabulary = get_full_vocabulary(corpus)

    x_train, x_test, y_train, y_test = train_test_split(temp, y, test_size=test_size, random_state=random_state)
    tf_idf_vector = tf_idf_vectorize(x_train, x_train[corpus.name], vocabulary=vocabulary)[1].reset_index()
    x_train = x_train.reset_index()
    x_train = pd.concat((x_train, tf_idf_vector), axis=1).drop([corpus.name, "index"], axis=1)

    tf_idf_vector = tf_idf_vectorize(x_test, x_test[corpus.name], vocabulary=vocabulary)[1].reset_index()
    x_test = x_test.reset_index()
    x_test = pd.concat((x_test, tf_idf_vector), axis=1).drop([corpus.name, "index"], axis=1)

    return (x_train, x_test, y_train, y_test)




In [159]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["class"]
corpus = filtered["text"]



#just tweet sentiment classifier
tweet_features = ["text_NN_tweets", "text_log_tweets",  "text_tb_sub_class"]

#just imbd sentiment classifier
imdb_features = ["text_NN_imdb", "text_log_imdb"]

# both sentiment classifiers
tweet_and_imdb = ["text_NN_tweets", "text_log_tweets",  "text_tb_sub_class", 
                           "text_NN_imdb", "text_log_imdb"]

# raw polarity and subjectivity scores from Textblob, Vader
raw_features = ["text_tb_pol",	"text_tb_sub",
             		"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]

# dataframe to store accuracies for NN and log regression
accuracy_df = pd.DataFrame(columns=["tweet_classifier",
               "imdb_classifier",
               "combined_classifier",
               "raw_sentiments"])


EXCEL_FILE = r"../data/Des_fake_news/Sentiment_Analysis_Results/LIAR_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


In [161]:
'''
  Logistic Regression work
'''


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix




# average accuracies
logOutput = {}
logMatrices = {"tweet_classifier" : [],
               "imdb_classifier" : [],
               "combined_classifier": [],
               "raw_sentiments" : []}

# sums
tweet_scores = 0
imdb_scores = 0
tweet_imdb_scores = 0
raw_scores = 0

# log regression model, LBFGS with L2 penalty
lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")

log_combined_pred = None
log_raw_pred = None


# run 100 iterations
#for i in range(1):
  #print(f"Iteration {i}")
  #tweet_accuracy
x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[tweet_features], classes, corpus, test_size=0.15, random_state=42)


lbgfs.fit(x_train, y_train)
y_pred = lbgfs.predict(x_test)
tweet_scores += accuracy_score(y_test, y_pred)
logMatrices["tweet_classifier"].append(confusion_matrix(y_test, y_pred))

x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[imdb_features], classes, corpus, test_size=0.15, random_state=42)
lbgfs.fit(x_train, y_train)
y_pred = lbgfs.predict(x_test)
imdb_scores += accuracy_score(y_test, y_pred)
logMatrices["imdb_classifier"].append(confusion_matrix(y_test, y_pred))

x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[tweet_and_imdb], classes, corpus, test_size=0.15, random_state=42)
lbgfs.fit(x_train, y_train)
log_combined_pred = lbgfs.predict(x_test)
tweet_imdb_scores += accuracy_score(y_test, log_combined_pred)
logMatrices["combined_classifier"].append(confusion_matrix(y_test, log_combined_pred))

x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[raw_features], classes, corpus, test_size=0.15, random_state=42)
lbgfs.fit(x_train, y_train)
log_raw_pred = lbgfs.predict(x_test)
raw_scores += accuracy_score(y_test, log_raw_pred)
logMatrices["raw_sentiments"].append(confusion_matrix(y_test, log_raw_pred))


log_y_actual = y_test

logOutput["tweet_classifier"] = tweet_scores / 1
logOutput["imdb_classifier"] = imdb_scores / 1
logOutput["combined_classifier"] = tweet_imdb_scores / 1
logOutput["raw_sentiments"] = raw_scores / 1

logMatrices["tweet_classifier"] = np.mean(np.array(logMatrices["tweet_classifier"]), axis=0)
logMatrices["imdb_classifier"] = np.mean(np.array(logMatrices["imdb_classifier"]), axis=0)
logMatrices["combined_classifier"] = np.mean(np.array(logMatrices["combined_classifier"]), axis=0)
logMatrices["raw_sentiments"] = np.mean(np.array(logMatrices["raw_sentiments"]), axis=0)


# write confusion matrices and save
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book

for i in logMatrices.keys():
  pd.DataFrame(logMatrices[i]).to_excel(writer, sheet_name=f"matrix_log_{i}")

book.save(filename=EXCEL_FILE)
book.close()

accuracy_df.loc["Log Regression"] = logOutput


c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

In [162]:
'''
    Compile and save neural net models
'''

import tensorflow as tf

vocab = get_full_vocabulary(filtered["text"])

tweet_len = len(tweet_features) + len(vocab)
imdb_len = len(imdb_features) + len(vocab)
combined_len = len(tweet_and_imdb) + len(vocab)
raw_len = len(raw_features) + len(vocab)


tweet_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(tweet_len, 1)),
  tf.keras.layers.Dense(tweet_len, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax')
])
tweet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


imdb_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(imdb_len, 1)),
  tf.keras.layers.Dense(imdb_len, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax')
])
imdb_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


combined_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(combined_len, 1)),
  tf.keras.layers.Dense(combined_len, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax')
])
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



raw_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(raw_len, 1)),
  tf.keras.layers.Dense(raw_len, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax')
])

raw_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])






c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
'''
    NN results
'''

EPOCHS = 100
nnOutput = {}
nnMatrices = {"tweet_classifier" : None,
               "imdb_classifier" : None,
               "combined_classifier": None,
               "raw_sentiments" : None}



# tweet NN
x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[tweet_features], classes, corpus, test_size=0.15, random_state=42)


tweet_model.fit(x_train, y_train, epochs=1, verbose=2)
model_loss1, model_acc1 = tweet_model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = tweet_model.evaluate(x_test,  y_test, verbose=2)
nnOutput["tweet_classifier"] = model_acc2
tweet_pred = [1 if i >= 0.5 else 0 for i in tweet_model.predict(x_test)]
nnMatrices["tweet_classifier"] = confusion_matrix(y_test, tweet_pred)

# imdb NN
x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[imdb_features], classes, corpus, test_size=0.15, random_state=42)
imdb_model.fit(x_train, y_train, epochs=1, verbose=2)
model_loss1, model_acc1 = imdb_model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = imdb_model.evaluate(x_test,  y_test, verbose=2)
nnOutput["imdb_classifier"] = model_acc2
imdb_pred = [1 if i >= 0.5 else 0 for i in imdb_model.predict(x_test)]
nnMatrices["imdb_classifier"] = confusion_matrix(y_test, imdb_pred)

# combined NN
x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[tweet_and_imdb], classes, corpus, test_size=0.15, random_state=42)
combined_model.fit(x_train, y_train, epochs=1, verbose=2)
model_loss1, model_acc1 = combined_model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = combined_model.evaluate(x_test,  y_test, verbose=2)
nnOutput["combined_classifier"] = model_acc2
combined_pred = [1 if i >= 0.5 else 0 for i in combined_model.predict(x_test)]
nnMatrices["combined_classifier"] = confusion_matrix(y_test, combined_pred)

# raw NN
x_train, x_test, y_train, y_test = train_test_split_and_vectorize(filtered[raw_features], classes, corpus, test_size=0.15, random_state=42)
raw_model.fit(x_train, y_train, epochs=EPOCHS, verbose=2)
model_loss1, model_acc1 = raw_model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = raw_model.evaluate(x_test,  y_test, verbose=2)
nnOutput["raw_sentiments"] = model_acc2
raw_pred = [1 if i >= 0.5 else 0 for i in raw_model.predict(x_test)]
nnMatrices["raw_sentiments"] = confusion_matrix(y_test, raw_pred)


# write confusion matrices and save
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book

for i in nnMatrices.keys():
  pd.DataFrame(nnMatrices[i]).to_excel(writer, sheet_name=f"matrix_NN_{i}")

book.save(filename=EXCEL_FILE)
book.close()
accuracy_df.loc["Neural Net"] = nnOutput



,text_NN_tweets,text_log_tweets,text_tb_sub_class,__word0,__word1,__word2,__word3,__word4,__word5,__word6,...,__word327,__word328,__word329,__word330,__word331,__word332,__word333,__word334,__word335,__word336
0,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.270349,0.000000,0.0,0.0,0.0
4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7192,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
7193,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
7194,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.413479,0.0,0.0,0.0
7195,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [8]:

model_loss1, model_acc1 = raw_model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = raw_model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

225/225 - 0s - 1ms/step - accuracy: 0.0624 - loss: -4.8417e+03
40/40 - 0s - 2ms/step - accuracy: 0.0732 - loss: -4.8668e+03
Train / Test Accuracy: 6.2% / 7.3%


In [ ]:
'''
	Categorical feature sets for Naive Bayes and Random Forests
'''


# both sentiment classifiers
categorical_tweet_and_imdb = filtered[["text_tb_sub_class",  "text_NN_imdb", "text_NN_tweets",	
                                       	 "text_log_imdb", "text_log_tweets",  ]].to_numpy()

# raw polarity and subjectivity scores from Textblob, Vader
categorical_raw_features = filtered[["text_vader_class", "text_tb_pol_class", "text_tb_sub_class"]].to_numpy()

In [ ]:
'''
  Naive Bayes work
'''


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix




# average accuracies
nbOutput = {}
nbMatrices = {"combined_classifier": None,
               "raw_sentiments" : None}

# sums
tweet_scores = 0
imdb_scores = 0
tweet_imdb_scores = 0
raw_scores = 0

# log regression model, LBFGS with L2 penalty
nb = CategoricalNB()

nb_combined_pred = None
nb_raw_pred = None

# run 100 iterations
#for i in range(1):
  #print(f"Iteration {i}")
  #tweet_accuracy

x_train, x_test, y_train, y_test = train_test_split(categorical_tweet_and_imdb, classes, test_size=0.15, random_state=42)
nb.fit(x_train, y_train)
nb_combined_pred = nb.predict(x_test)
tweet_imdb_scores += accuracy_score(y_test, nb_combined_pred)
nbMatrices["combined_classifier"] = confusion_matrix(y_test, nb_combined_pred)

x_train, x_test, y_train, y_test = train_test_split(categorical_raw_features, classes, test_size=0.15, random_state=42)
nb.fit(x_train, y_train)
nb_raw_pred = nb.predict(x_test)
raw_scores += accuracy_score(y_test, nb_raw_pred)
nbMatrices["raw_sentiments"] = confusion_matrix(y_test, nb_raw_pred)


log_y_actual = y_test

nbOutput["tweet_classifier"] = 0
nbOutput["imdb_classifier"] = 0
nbOutput["combined_classifier"] = tweet_imdb_scores / 1
nbOutput["raw_sentiments"] = raw_scores / 1


nbMatrices["combined_classifier"] = np.mean(np.array(nbMatrices["combined_classifier"]), axis=0)
nbMatrices["raw_sentiments"] = np.mean(np.array(nbMatrices["raw_sentiments"]), axis=0)


# write confusion matrices and save
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book

for i in nbMatrices.keys():
  pd.DataFrame(nbMatrices[i]).to_excel(writer, sheet_name=f"matrix_nb_{i}")

book.save(filename=EXCEL_FILE)
book.close()

accuracy_df.loc["Naive Bayes"] = nbOutput

accuracy_df


In [ ]:
'''
    Random forests work
'''

from sklearn.ensemble import RandomForestClassifier

# average accuracies
rfOutput = {}
rfMatrices = {"combined_classifier": None,
               "raw_sentiments" : None}

# sums
tweet_scores = 0
imdb_scores = 0
tweet_imdb_scores = 0
raw_scores = 0

# log regression model, LBFGS with L2 penalty
rf = RandomForestClassifier()

rf_combined_pred = None
rf_raw_pred = None

# run 100 iterations
#for i in range(1):
  #print(f"Iteration {i}")
  #tweet_accuracy

x_train, x_test, y_train, y_test = train_test_split(categorical_tweet_and_imdb, classes, test_size=0.15, random_state=42)
rf.fit(x_train, y_train)
rf_combined_pred = rf.predict(x_test)
tweet_imdb_scores += accuracy_score(y_test, rf_combined_pred)
rfMatrices["combined_classifier"] = confusion_matrix(y_test, rf_combined_pred)

x_train, x_test, y_train, y_test = train_test_split(categorical_raw_features, classes, test_size=0.15, random_state=42)
rf.fit(x_train, y_train)
rf_raw_pred = rf.predict(x_test)
raw_scores += accuracy_score(y_test, rf_raw_pred)
rfMatrices["raw_sentiments"] = confusion_matrix(y_test, rf_raw_pred)


log_y_actual = y_test

rfOutput["tweet_classifier"] = 0
rfOutput["imdb_classifier"] = 0
rfOutput["combined_classifier"] = tweet_imdb_scores / 1
rfOutput["raw_sentiments"] = raw_scores / 1


rfMatrices["combined_classifier"] = np.mean(np.array(rfMatrices["combined_classifier"]), axis=0)
rfMatrices["raw_sentiments"] = np.mean(np.array(rfMatrices["raw_sentiments"]), axis=0)


# write confusion matrices and save
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book

for i in rfMatrices.keys():
  pd.DataFrame(rfMatrices[i]).to_excel(writer, sheet_name=f"matrix_rf_{i}")

book.save(filename=EXCEL_FILE)
book.close()

accuracy_df.loc["Random Forest"] = rfOutput

accuracy_df



In [ ]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''

book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
accuracy_df.to_excel(writer, sheet_name=f"predicion_accuracies")
book.save(filename=EXCEL_FILE)
book.close()



In [ ]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io

y = y_test
log_y = log_y_actual

raw_scores = np.array(raw_pred)
combined_scores = np.array(combined_pred)

raw_fpr, raw_tpr, raw_thresh = metrics.roc_curve(y, raw_scores, pos_label=1)
raw_roc_auc = metrics.auc(raw_fpr, raw_tpr)

com_fpr, com_tpr, com_thresh = metrics.roc_curve(y, combined_scores, pos_label=1)
com_roc_auc = metrics.auc(com_fpr, com_tpr)

raw_fpr_log, raw_tpr_log, raw_thresh_log = metrics.roc_curve(y, log_raw_pred, pos_label=1)
raw_roc_auc_log = metrics.auc(raw_fpr_log, raw_tpr_log)

com_fpr_log, com_tpr_log, com_thresh_log = metrics.roc_curve(y, log_combined_pred, pos_label=1)
com_roc_auc_log = metrics.auc(com_fpr_log, com_tpr_log)

raw_fpr_nb, raw_tpr_nb, raw_thresh_nb = metrics.roc_curve(y, nb_raw_pred, pos_label=1)
raw_roc_auc_nb = metrics.auc(raw_fpr_nb, raw_tpr_nb)

com_fpr_nb, com_tpr_nb, com_thresh_nb = metrics.roc_curve(y, nb_combined_pred, pos_label=1)
com_roc_auc_nb = metrics.auc(com_fpr_nb, com_tpr_nb)

raw_fpr_rf, raw_tpr_rf, raw_thresh_rf = metrics.roc_curve(y, rf_raw_pred, pos_label=1)
raw_roc_auc_rf = metrics.auc(raw_fpr_rf, raw_tpr_rf)

com_fpr_rf, com_tpr_rf, com_thresh_rf = metrics.roc_curve(y, rf_combined_pred, pos_label=1)
com_roc_auc_rf = metrics.auc(com_fpr_rf, com_tpr_rf)


plt.figure()
lw = 2
plt.plot(raw_fpr, raw_tpr,
 lw=lw, label='Raw NN (%0.2f)' % raw_roc_auc)
plt.plot(com_fpr, com_tpr,
 lw=lw, label='Sentiment Classifier NN  (%0.2f)' % com_roc_auc)
plt.plot(raw_fpr_log, raw_tpr_log,
 lw=lw, label='Raw Log (%0.2f)' % raw_roc_auc_log)
plt.plot(com_fpr_log, com_tpr_log,
 lw=lw, label='Sentiment Classifier Log  (%0.2f)' % com_roc_auc_log)

plt.plot(raw_fpr_nb, raw_tpr_nb,
 lw=lw, label='Raw Naive Bayes (%0.2f)' % raw_roc_auc_nb)
plt.plot(com_fpr_nb, com_tpr_nb,
 lw=lw, label='Sentiment Classifier Naive Bayes  (%0.2f)' % com_roc_auc_nb)

plt.plot(raw_fpr_rf, raw_tpr_rf,
 lw=lw, label='Raw Naive Random Forest (%0.2f)' % raw_roc_auc_rf)
plt.plot(com_fpr_rf, com_tpr_rf,
 lw=lw, label='Sentiment Classifier Random Forest  (%0.2f)' % com_roc_auc_rf)




plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()

